# CC6204 Deep Learning, Universidad de Chile
## Código de Red Neuronal estilo pytorch
El siguiente código muestra la manera estándard de crear una red neuronal Feed Forward de dos capas escondidas usando todas las funcionalidades de pytorch. La idea es que el código sirva para aprender la arquitectura general de red+entrenamiento+predicción usando las clases abstractas de pytorch además de las utilidades que entrega para crear capaz escondidas con muy poco esfuerzo. Al igual como las capas lineales creadas en este ejemplo, se pueden agregar, por ejemplo, capas convolucionales que empezaremos a ver en la segunda parte del curso.

Código por Jorge Pérez

https://github.com/jorgeperezrojas

[@perez](https://twitter.com/perez)

In [1]:
# Esto es solo para poder debugear.
# !pip install -q ipdb

import torch
import numpy as np
import sys
import time
# import ipdb

# Genera una semilla fija para que los experimentos sea repetibles.
t_cg = torch.manual_seed(1547)

In [2]:
# Red estilo pytorch
class FFNN(torch.nn.Module):
  def __init__(self, d0=300, d1=200, d2=300):
    super(FFNN, self).__init__()

    # Definimos capas (automáticamente se registran como parametros)
    self.fc1 = torch.nn.Linear(d0, d1, bias=True)
    self.fc2 = torch.nn.Linear(d1, d2, bias=True)
    self.fc3 = torch.nn.Linear(d2,  1, bias=True)

  # Computa la pasada hacia adelante
  def forward(self, x):

    u1 = self.fc1(x)
    h1 = torch.tanh(u1)
    u2 = self.fc2(h1)
    h2 = torch.sigmoid(u2)
    u3 = self.fc3(h2)
    y_pred = torch.sigmoid(u3)

    return y_pred

In [3]:
from torch.utils.data import Dataset, DataLoader

class RandomDataSet(Dataset):
  def __init__(self, N, f):
    R_N_f = torch.rand(N,f)
    self.X = torch.bernoulli(R_N_f)
    R_N_1 = torch.rand(N,1)
    self.Y = torch.bernoulli(R_N_1)
    self.num_features = f

  # Debemos definir __len__ para retornar el tamaño del dataset
  def __len__(self):
    return self.X.size()[0]

  # Debemos definir __getitem__ para retornar el i-ésimo
  # ejemplo en nuestro dataset.
  def __getitem__(self, i):
    return self.X[i], self.Y[i]

In [15]:
def loop_FFNN(dataset, batch_size, d1, d2, lr,
                 epochs, run_in_GPU=True, reports_every=1,
                 cheq_grad=False):

  # Define un tipo para los tensores según si correrá en la GPU o no
  device = 'cuda' if run_in_GPU else 'cpu'

  # d0 es la cantidad de features
  d0 = dataset.num_features

  # Crea la red
  red = FFNN(d0, d1, d2)

  # Pasa la red al dispositivo elegido
  red.to(device)

  # Muestra la cantidad de parámetros
  print('Red:', red)

  # Crea un dataloader desde el dataset
  data = DataLoader(dataset, batch_size, shuffle=True)

  # Crea un optimizador para el descenso de gradiente
  optimizador = torch.optim.SGD(red.parameters(), lr)

  # Define una perdida
  perdida = torch.nn.BCELoss()

  # Comienza el entrenamiento
  tiempo_epochs = 0
  for e in range(1,epochs+1):
    inicio_epoch = time.time()

    for (x,y) in data:
      # Asegura de pasarlos a la GPU si fuera necesario
      x, y = x.to(device), y.to(device)

      # Computa la pasada hacia adelante (forward)
      y_pred = red.forward(x)

      # Computa la función de pérdida
      L = perdida(y_pred,y)

      # Computa los gradientes hacia atrás (backpropagation)
      L.backward()

      # Descenso de gradiente para actualizar los parámetros
      optimizador.step()

      # Limpia los gradientes
      optimizador.zero_grad()

    tiempo_epochs += time.time() - inicio_epoch

    if e % reports_every == 0:
      # Calcula la certeza de las predicciones sobre todo el conjunto
      X = dataset.X.to(device)
      Y = dataset.Y.to(device)

      # Predice usando la red
      Y_PRED = red.forward(X)

      # Calcula la pérdida de todo el conjunto
      L_total = perdida(Y_PRED, Y)

      # Elige una clase dependiendo del valor de Y_PRED
      Y_PRED_BIN = (Y_PRED >= 0.5).float()

      correctos = torch.sum(Y_PRED_BIN == Y).item()
      acc = (correctos / N) * 100

      sys.stdout.write(
            '\rEpoch:{0:03d}'.format(e) + ' Acc:{0:.2f}%'.format(acc)
            + ' Loss:{0:.4f}'.format(L_total)
            + ' Tiempo/epoch:{0:.3f}s'.format(tiempo_epochs/e))

In [5]:
N = 5000 # numero de ejemplos
f = 300 # numero de features

dataset = RandomDataSet(N,f)

In [16]:
epochs = 50
loop_FFNN(dataset, batch_size=10, d1=300, d2=400, epochs=epochs,
             run_in_GPU=True, lr=0.05)

Red: FFNN(
  (fc1): Linear(in_features=300, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=1, bias=True)
)
Epoch:050 Acc:99.42% Loss:0.0583 Tiempo/epoch:0.331s